In [42]:
import math
import matplotlib.pyplot as plt
from scipy.special import gamma
import numpy as np

In [ ]:
#Block with additional feature - not yet incorporated into main.py

#Enumeration - mass ordering
massOrdering = ['NMO', 'IMO']
enumMassOrdering = enumerate(massOrdering)

#Enumeration - progenitor
progenitor = ['ONeMg', 'Fe']
enumProgenitor = enumerate(progenitor)

#progenitor type, prompt user to enter either 'ONeMg' or 'Fe', still need to make case insensitive 
p = raw_input("What is the progenitor type? Please enter either 'ONeMg' or 'Fe': ")

#compactness, prompt user to enter either 2.5 or 5 
comp = float(raw_input("What is the compactness? Please enter either 2.5 or 5: "))

#accretion timescale - units? 
timeAcc = float(raw_input("What is the accretion timescale? "))

#stiffness parameter prompt 
stiff = float(raw_input("What is the stiffness parameter? "))

#prompt user to enter either NMO or IMO, still need to make case insensitive 
mass_ordering = raw_input("What is the mass ordering? Please enter either NMO or IMO: ")

In [43]:
#From the numericalunits package: https://pypi.org/project/numericalunits/

#Set all variables to help introspection libraries
m = kg = s = C = K = 0.

cm = mm = um = nm = pm = fm = km = angstrom = lightyear = \
    astro_unit = pc = kpc = Mpc = Gpc = inch = foot = mile = thou = 0.
    
ms = us = ns = ps = fs = minute = hour = day = week = year = 0.
    
J = mJ = uJ = nJ = pJ = fJ = kJ = MJ = GJ = erg = eV = meV = keV = MeV = GeV = \
    TeV = btu = smallcal = kcal = Wh = kWh = 0.
    
def reset_units(seed=None):
    
    #Set all units to new self-consistent, floating-point values. 
    
    """
    reset_units() --> units are randomized. This is the suggested use. Run this 
    before your calculation, display the final answer, then re-run this, then re-display
    the final answer. If you get the same answers both times, then your calculations
    are almost guaranteed to be free of dimensional-analysis-violating errors.
    This method is run automatically the first time this module is imported.
    
    reset_unit('SI') --> Set units so that all values are given in standard SI
    units (meters-kilograms-seconds) by default. In this mode, there is no way to test
    for dimensional-analysis-violating errors.
    
    reset_units(x) --> If you pass any other argument x, it's used as the seed
    for the random number generator.
    
    """
    
    import random
    
    global m, kg, s, C, K
    
    if seed == 'SI':
        m = 1.
        kg = 1.
        s = 1.
        C = 1.
        K = 1.
    else: 
        prior_random_state = random.getstate()
        
        if seed is Non: 
            random.seed()
        else:
            random.seed(seed)
        
        m = 10. ** random.uniform(-1, 1) #meter
        kg = 10. ** random.uniform(-1, 1) #kilogram
        s = 10. ** random.uniform(-1, 1) #second
        C = 10. ** random.uniform(-1, 1) #coulombs
        K = 10. ** random.uniform(-1, 1) #kelvins
        
        #Leave the random generator like found, in case someone else is using it.
        random.setstate(prior_random_state)
    
    set_derived_units_and_constants()
    return

def set_derived_units_and_constants():
    
    """
    Assuming that the base units (m, kg, s, C, K) have already been set as 
    floating point values, this function sets all other units and constants to the
    appropriate, self-consistent values.
    
    """
    
    #Length
    global cm, mm, km, lightyear, astro_unit, pc, kpc, Mpc, Gpc, inch, foot, mile
        
    cm = 1e-2 * m
    mm = 1e-3 * m
    km = 1e3 * m
    lightyear = 9460730472580800. * m
    astro_unit = 149597870700. * m #astronomical unit
    pc = (648000./math.pi) * astro_unit #parsec
    kpc = 1e3 * pc
    Mpc = 1e6 * pc
    Gpc = 1e9 * pc
    inch = 2.54 * cm
    foot = 12. * inch
    mile = 5280. * foot
    
    #Time
    global minute, hour, day, week, year
    
    minute = 60. * s
    hour = 60. * minute
    day = 24. * hour #solar day
    week = 7. * day
    year = 365.256363004 * day #sidereal year
    
    #Energy
    global J, erg, eV, keV, MeV, GeV, Tev
    
    J = (kg * m**2) / s**2
    erg = 1e-7 * J
    eV = 1.6021766208e-19 * J
    keV = 1e3 * eV
    Mev = 1e6 * eV
    GeV = 1e9 * eV
    TeV = 1e12 * eV

In [44]:
reset_units('SI')
set_derived_units_and_constants()

#Distance between supernova and detector on Earth (divide flux by 4*pi*dist^2)
#Using 51.4 kpc for SN1987A
dist = 10 * kpc

#Enumeration of matter type and flavor of neutrino
matterType = ['nu', 'nubar']
flavor = ['e', 'mu', 'tau']

#Luminosity
L_array = [[1.6e52 * (erg / s), 1.6e52 * (erg / s)],
            [1.6e52 * (erg / s), 1.6e52 * (erg / s)],
            [1.6e52 * (erg / s), 1.6e52 * (erg / s)]]
        
#Mean energy, 
meanE_array = [[0.015 * GeV, 0.015 * GeV],
                [0.015 * GeV, 0.015 * GeV],
                [0.015 * GeV, 0.015 * GeV]]
        
#Alpha (pinch parameters)
alpha_array = [[3, 4.31],
                [6, 6],
                [6, 6]]

#Creating a dictionary that is enumerated
alpha = {}
L = {}
meanE = {}
for x, pair_item in enumerate(flavor):
    for y, item in enumerate(matterType):
        alpha[(item, pair_item)] = alpha_array[x][y]
        L[(item, pair_item)] = L_array[x][y]
        meanE[(item, pair_item)] = meanE_array[x][y] 
                                
#Float range
def frange(start, stop, step = 0.0002):
    while start < stop:
        yield start
        start += step
        
def nuE(E):
    #Ratio of mean energy and luminosity
    ratio_e = (L[('nu', 'e')] / meanE[('nu', 'e')])
            
    #Numerator
    num_e = (alpha[('nu', 'e')] + 1) ** (alpha[('nu', 'e')] + 1)
            
    #Denominator
    den_e = (meanE[('nu', 'e')]) * gamma(alpha[('nu', 'e')] + 1)
            
    #Fraction
    frac_e = (num_e / den_e)
      
    flux_e = ((ratio_e) * (frac_e) * ((E / meanE[('nu', 'e')]) ** (alpha[('nu', 'e')])) * math.exp(-((alpha[('nu', 'e')] + 1) * E) / meanE[('nu', 'e')])) / (4*math.pi*dist**2)
    return flux_e

def nubarE(E):
    #Ratio of mean energy and luminosity
    ratio_anti_e = (L[('nubar', 'e')] / meanE[('nubar', 'e')])
            
    #Numerator
    num_anti_e = (alpha[('nubar', 'e')] + 1) ** (alpha[('nubar', 'e')] + 1)  
            
    #Denominator
    den_anti_e = (meanE[('nubar', 'e')]) * gamma(alpha[('nubar', 'e')] + 1)
            
    #Fraction
    frac_anti_e = (num_anti_e / den_anti_e)

    flux_anti_e = ((ratio_anti_e) * (frac_anti_e) * ((E / meanE[('nubar', 'e')]) ** (alpha[('nubar', 'e')])) * math.exp(-((alpha[('nubar', 'e')] + 1) * E) / meanE[('nubar', 'e')])) / (4*math.pi*dist**2)
    return flux_anti_e
        
def nuMu(E):
    #Ratio of mean energy and luminosity
    ratio_mu = (L[('nu', 'mu')] / meanE[('nu', 'mu')])
            
    #Numerator
    num_mu = (alpha[('nu', 'mu')] + 1) ** (alpha[('nu', 'mu')] + 1)
            
    #Denominator
    den_mu = (meanE[('nu', 'mu')]) * gamma(alpha[('nu', 'mu')] + 1)
            
    #Fraction
    frac_mu = (num_mu / den_mu)

    flux_mu = ((ratio_mu) * (frac_mu) * ((E / meanE[('nu', 'mu')]) ** (alpha[('nu', 'mu')])) * math.exp(-((alpha[('nu', 'mu')] + 1) * E) / meanE[('nu', 'mu')])) / (4*math.pi*dist**2)
    return flux_mu
              
def nubarMu(E):
    #Ratio of mean energy and luminosity
    ratio_anti_mu = (L[('nubar', 'mu')] / meanE[('nubar', 'mu')])
            
    #Numerator
    num_anti_mu = (alpha[('nubar', 'mu')] + 1) ** (alpha[('nubar', 'mu')] + 1)
            
    #Denominator
    den_anti_mu = (meanE[('nubar', 'mu')]) * gamma(alpha[('nubar', 'mu')] + 1)
            
    #Fraction
    frac_anti_mu = (num_anti_mu / den_anti_mu)
      
    flux_anti_mu = ((ratio_anti_mu) * (frac_anti_mu) * ((E / meanE[('nubar', 'mu')]) ** (alpha[('nubar', 'mu')])) * math.exp(-((alpha[('nubar', 'mu')] + 1) * E) / meanE[('nubar', 'mu')])) / (4*math.pi*dist**2)
    return flux_anti_mu
                
def nuTau(E):
    #Ratio of mean energy and luminosity
    ratio_tau = (L[('nu', 'tau')] / meanE[('nu', 'tau')])
            
    #Numerator
    num_tau = (alpha[('nu', 'tau')] + 1) ** (alpha[('nu', 'tau')] + 1)  
            
    #Denominator
    den_tau = (meanE[('nu', 'tau')]) * gamma(alpha[('nu', 'tau')] + 1)
            
    #Fraction
    frac_tau = (num_tau / den_tau)
    
    flux_tau = ((ratio_tau) * (frac_tau) * ((E / meanE[('nu', 'tau')]) ** (alpha[('nu', 'tau')])) * math.exp(-((alpha[('nu', 'tau')] + 1) * E) / meanE[('nu', 'tau')])) / (4*math.pi*dist**2)
    return flux_tau
                
def nubarTau(E):
    #Ratio of mean energy and luminosity
    ratio_anti_tau = (L[('nubar', 'tau')] / meanE[('nubar', 'tau')])
    
    #Numerator
    num_anti_tau = (alpha[('nubar', 'tau')] + 1) ** (alpha[('nubar', 'tau')] + 1)
            
    #Denominator
    den_anti_tau = (meanE[('nubar', 'tau')]) * gamma(alpha[('nubar', 'tau')] + 1)
            
    #Fraction
    frac_anti_tau = (num_anti_tau / den_anti_tau)
    
    flux_anti_tau = ((ratio_anti_tau) * (frac_anti_tau) * ((E / meanE[('nubar', 'tau')]) ** (alpha[('nubar', 'tau')])) * math.exp(-((alpha[('nubar', 'tau')] + 1) * E) / meanE[('nubar', 'tau')])) / (4*math.pi*dist**2)
    return flux_anti_tau

In [45]:
#Enumeration of matter type and flavor of neutrino
matterType = ['nu', 'nubar']
flavor = ['e', 'mu', 'tau']

#Luminosity
L_array = [[1.6e52 * (erg / s), 1.6e52 * (erg / s)],
            [1.6e52 * (erg / s), 1.6e52 * (erg / s)],
            [1.6e52 * (erg / s), 1.6e52 * (erg / s)]]
        
#Mean energy, 
meanE_array = [[0.015 * GeV, 0.015 * GeV],
                [0.015 * GeV, 0.015 * GeV],
                [0.015 * GeV, 0.015 * GeV]]
        
#Alpha (pinch parameters)
alpha_array = [[3, 4.31],
                [6, 6],
                [6, 6]]

#Creating a dictionary that is enumerated
alpha = {}
L = {}
meanE = {}
for x, pair_item in enumerate(flavor):
    for y, item in enumerate(matterType):
        alpha[(item, pair_item)] = alpha_array[x][y]
        L[(item, pair_item)] = L_array[x][y]
        meanE[(item, pair_item)] = meanE_array[x][y] 
        

print((L[('nu', 'e')] / meanE[('nu', 'e')]))
print(L[('nubar', 'e')] / meanE[('nubar', 'e')])
print(L[('nu', 'mu')] / meanE[('nu', 'mu')])
print(L[('nubar', 'mu')] / meanE[('nubar', 'mu')])
print(L[('nu', 'tau')] / meanE[('nu', 'tau')])
print(L[('nubar', 'tau')] / meanE[('nubar', 'tau')])

print((alpha[('nu', 'e')] + 1) ** (alpha[('nu', 'e')] + 1))
print((alpha[('nubar', 'e')] + 1) ** (alpha[('nubar', 'e')] + 1))
print((alpha[('nu', 'mu')] + 1) ** (alpha[('nu', 'mu')] + 1))
print((alpha[('nubar', 'mu')] + 1) ** (alpha[('nubar', 'mu')] + 1))
print((alpha[('nu', 'tau')] + 1) ** (alpha[('nu', 'tau')] + 1))
print((alpha[('nubar', 'tau')] + 1) ** (alpha[('nubar', 'tau')] + 1))

print((meanE[('nu', 'e')]) * gamma(alpha[('nu', 'e')] + 1))
print((meanE[('nubar', 'e')]) * gamma(alpha[('nubar', 'e')] + 1))
print((meanE[('nu', 'mu')]) * gamma(alpha[('nu', 'mu')] + 1))
print((meanE[('nubar', 'mu')]) * gamma(alpha[('nubar', 'mu')] + 1))
print((meanE[('nu', 'tau')]) * gamma(alpha[('nu', 'tau')] + 1))
print((meanE[('nubar', 'tau')]) * gamma(alpha[('nubar', 'tau')] + 1))

6.65760973428e+56
6.65760973428e+56
6.65760973428e+56
6.65760973428e+56
6.65760973428e+56
6.65760973428e+56
256
7083.56538369
823543
823543
823543
823543
1.4419589587200001e-11
9.296087926989177e-11
1.730350750464e-09
1.730350750464e-09
1.730350750464e-09
1.730350750464e-09


In [46]:
#File-output for energies of neutrinos (tab delimited) - will output from for loop
#Will also output graphs of neutrino spectra
#Can change start, stop, and step later - first testing to see if for loop works and writes a text file

for t in range(0, 20, 1):
    #Functions go here
    file = open("data/data_%d.txt" % t, "w+")
    for E in frange(0, 0.1002 , 0.0002):
        flux_e = (nuE(E * GeV) * (s * cm**2) * 0.0002 * GeV) 
        flux_anti_e = (nubarE(E * GeV) * (s * cm**2) * 0.0002 * GeV)
        flux_mu = (nuMu(E * GeV) * (s * cm**2) * 0.0002 * GeV)
        flux_anti_mu = (nubarMu(E * GeV) * (s * cm**2) * 0.0002 * GeV)
        flux_tau = (nuTau(E * GeV) * (s * cm**2) * 0.0002 * GeV)
        flux_anti_tau = (nubarTau(E * GeV) * (s * cm**2) * 0.0002 * GeV)
        print(str(E)+ '\t' + str(flux_e) + '\t' + str(flux_anti_e) + '\t' + str(flux_mu) + '\t' + str(flux_anti_mu) + '\t' + str(flux_tau) + '\t' + str(flux_anti_tau))
        stringData = str(E)+ '\t' + str(flux_e) + '\t' + str(flux_anti_e) + '\t' + str(flux_mu) + '\t' + str(flux_anti_mu) + '\t' + str(flux_tau) + '\t' + str(flux_anti_tau)
        file = open("data/data_%d.txt" % t, "a") 
        file.write(stringData + '\n')
        file.close()
    file = open("data/data_%d.txt" %t)
    lines = file.readlines()
    E = []
    flux_e = []
    flux_anti_e = []
    flux_mu = []
    flux_anti_mu = []
    flux_tau = []
    flux_anti_tau = []
    for line in lines:
        E.append(line.split()[0])
    for line in lines:
        flux_e.append(line.split()[1])
    for line in lines:
        flux_anti_e.append(line.split()[2])   
    for line in lines:
        flux_mu.append(line.split()[3])
    for line in lines:
        flux_anti_mu.append(line.split()[4])
    for line in lines:
        flux_tau.append(line.split()[5])
    for line in lines:
        flux_anti_tau.append(line.split()[6])
    file.close()
    plt.figure()
    plt.plot(E, flux_e, 'pink', lw = 2, label="Electron neutrino")
    plt.plot(E, flux_anti_e, 'mediumpurple', lw = 2, label="Electron antineutrino")
    plt.plot(E, flux_mu, 'lightskyblue', lw = 2, label="Mu neutrino")
    plt.plot(E, flux_anti_mu, 'silver', lw = 2, label="Mu antineutrino")
    plt.plot(E, flux_tau, 'gold', lw = 2, label="Tau neutrino")
    plt.plot(E, flux_anti_tau, 'red', lw = 2, label="Tau antineutrino")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Neutrino Spectra_%d' %t)
    plt.xlabel("Energy")
    plt.ylabel("Flux")
    plt.grid()
    plt.savefig('spectra/Neutrino Spectra_%d.png' %t, bbox_inches='tight')

0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	71135.87337728914	1049.1494429250065	4.343083380830239	4.343083380830239	4.343083380830239	4.343083380830239
0.0004	539530.8502579218	19387.80247162643	253.18850802589574	253.18850802589574	253.18850802589574	253.18850802589574
0.0006	1726345.3820654785	103689.28113323684	2626.983786013153	2626.983786013153	2626.983786013153	2626.983786013153
0.0008	3879551.530953204	333788.80678547476	13444.840681545975	13444.840681545975	13444.840681545975	13444.840681545975
0.001	7183716.609421044	813590.5129891956	46717.71432846784	46717.71432846784	46717.71432846784	46717.71432846784
0.0012	11768755.959500786	1663140.7202343794	127067.62461033904	127067.62461033904	127067.62461033904	127067.62461033904
0.0014	17717749.362329878	3011075.2623538887	291864.6500615481	291864.6500615481	291864.6500615481	291864.6500615481
0.0016	25073907.67272573	4987912.8074352145	592378.1453690883	592378.1453690883	592378.1453690883	592378.1453690883
0.0018	33846768.99477342	7720371.0

0.0984	35.91361672582025	0.33567170289150117	0.0007714722810569423	0.0007714722810569423	0.0007714722810569423	0.0007714722810569423
0.0986	34.25643940569589	0.31547671621706924	0.0007113397433978906	0.0007113397433978906	0.0007113397433978906	0.0007113397433978906
0.0988	32.67532660372667	0.2964914609665971	0.0006558780550869053	0.0006558780550869053	0.0006558780550869053	0.0006558780550869053
0.099	31.166807233828923	0.2786438090678017	0.0006047257315606814	0.0006047257315606814	0.0006047257315606814	0.0006047257315606814
0.0992	29.72756758847526	0.26186591126551023	0.0005575491554084373	0.0005575491554084373	0.0005575491554084373	0.0005575491554084373
0.0994	28.354444254360477	0.24609394468173243	0.0005140404369555584	0.0005140404369555584	0.0005140404369555584	0.0005140404369555584
0.0996	27.044417344393082	0.23126787518008945	0.00047391543836939416	0.00047391543836939416	0.00047391543836939416	0.00047391543836939416
0.0998	25.794604032022132	0.21733123367212412	0.00043691194884790

0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	71135.87337728914	1049.1494429250065	4.343083380830239	4.343083380830239	4.343083380830239	4.343083380830239
0.0004	539530.8502579218	19387.80247162643	253.18850802589574	253.18850802589574	253.18850802589574	253.18850802589574
0.0006	1726345.3820654785	103689.28113323684	2626.983786013153	2626.983786013153	2626.983786013153	2626.983786013153
0.0008	3879551.530953204	333788.80678547476	13444.840681545975	13444.840681545975	13444.840681545975	13444.840681545975
0.001	7183716.609421044	813590.5129891956	46717.71432846784	46717.71432846784	46717.71432846784	46717.71432846784
0.0012	11768755.959500786	1663140.7202343794	127067.62461033904	127067.62461033904	127067.62461033904	127067.62461033904
0.0014	17717749.362329878	3011075.2623538887	291864.6500615481	291864.6500615481	291864.6500615481	291864.6500615481
0.0016	25073907.67272573	4987912.8074352145	592378.1453690883	592378.1453690883	592378.1453690883	592378.1453690883
0.0018	33846768.99477342	7720371.0

0.0946	87.90879608366802	1.087450738390665	0.003587942178675175	0.003587942178675175	0.003587942178675175	0.003587942178675175
0.0948	83.87288133419645	1.0223854946879252	0.003309897078364325	0.003309897078364325	0.003309897078364325	0.003309897078364325
0.095	80.02118793997796	0.9611948489737878	0.0030533173505244517	0.0030533173505244517	0.0030533173505244517	0.0030533173505244517
0.0952	76.3453606911831	0.9036492535499973	0.002816552511187477	0.002816552511187477	0.002816552511187477	0.002816552511187477
0.0954	72.83742062774289	0.8495326843952157	0.0025980784391348433	0.0025980784391348433	0.0025980784391348433	0.0025980784391348433
0.0956	69.48974823133736	0.7986418492650739	0.002396487745727906	0.002396487745727906	0.002396487745727906	0.002396487745727906
0.0958	66.29506736145805	0.7507854418541431	0.002210480875057733	0.002210480875057733	0.002210480875057733	0.002210480875057733
0.096	63.24642990294908	0.7057834393604081	0.0020388578793222156	0.0020388578793222156	0.0020388578

0.0394	15691424.166303037	7644852.713143689	2883379.47027898	2883379.47027898	2883379.47027898	2883379.47027898
0.0396	15104170.597668156	7279450.691171234	2707456.1711103558	2707456.1711103558	2707456.1711103558	2707456.1711103558
0.0398	14537782.54077228	6930751.834343177	2541877.40200563	2541877.40200563	2541877.40200563	2541877.40200563
0.04	13991573.353817489	6598038.13374286	2386063.317132048	2386063.317132048	2386063.317132048	2386063.317132048
0.0402	13464876.257706288	6280619.71468293	2239464.5027024196	2239464.5027024196	2239464.5027024196	2239464.5027024196
0.0404	12957043.9028756	5977833.900566544	2101560.5309017943	2101560.5309017943	2101560.5309017943	2101560.5309017943
0.0406	12467447.937612927	5689044.297346029	1971858.5713442361	1971858.5713442361	1971858.5713442361	1971858.5713442361
0.0408	11995478.57842269	5413639.898874702	1849892.0586355668	1849892.0586355668	1849892.0586355668	1849892.0586355668
0.041	11540544.182967069	5151034.213374777	1735219.4145774145	173521

...

0.0538	858676.7686710283	178892.63611059036	22550.657378546857	22550.657378546857	22550.657378546857	22550.657378546857
0.054	823193.2362708392	169351.8367055873	21003.610691023983	21003.610691023983	21003.610691023983	21003.610691023983
0.0542	789143.5318102669	160310.39417567622	19561.08627441205	19561.08627441205	19561.08627441205	19561.08627441205
0.0544	756471.3217024537	151742.75522385986	18216.145891968517	18216.145891968517	18216.145891968517	18216.145891968517
0.0546	725122.4061406858	143624.63844165026	16962.30243162203	16962.30243162203	16962.30243162203	16962.30243162203
0.0548	695044.6422364054	135932.97319222317	15793.491291998727	15793.491291998727	15793.491291998727	15793.491291998727
0.055	666187.8697264397	128645.84129595346	14704.04352891591	14704.04352891591	14704.04352891591	14704.04352891591
0.0552	638503.8391738515	121742.42139812725	13688.660658152312	13688.660658152312	13688.660658152312	13688.660658152312
0.0554	611946.1425884216	115202.93590326706	12742.39101

0.0884	374.7565754225895	7.289833106525953	0.04312915091057136	0.04312915091057136	0.04312915091057136	0.04312915091057136
0.0886	357.7101471237194	6.858033709736743	0.039822222796657515	0.039822222796657515	0.039822222796657515	0.039822222796657515
0.0888	341.43388461510574	6.4516694360086815	0.03676772929705239	0.03676772929705239	0.03676772929705239	0.03676772929705239
0.089	325.89325309528886	6.069251078544506	0.03394649213604735	0.03394649213604735	0.03394649213604735	0.03394649213604735
0.0892	311.05525321825144	5.709376045644709	0.03134078266707896	0.03134078266707896	0.03134078266707896	0.03134078266707896
0.0894	296.88835348204896	5.37072335971966	0.02893421289887402	0.02893421289887402	0.02893421289887402	0.02893421289887402
0.0896	283.36242556171806	5.052048942716176	0.026711634665161885	0.026711634665161885	0.026711634665161885	0.026711634665161885
0.0898	270.4486824598803	4.7521811717034765	0.024659046333285802	0.024659046333285802	0.024659046333285802	0.024659046333285802

0.0844	947.6920858395679	24.60397373947795	0.2112491794849559	0.2112491794849559	0.2112491794849559	0.2112491794849559
0.0846	904.8750856122606	23.15727884846893	0.19517688283434953	0.19517688283434953	0.19517688283434953	0.19517688283434953
0.0848	863.9780837474227	21.795123510051347	0.1803213546324716	0.1803213546324716	0.1803213546324716	0.1803213546324716
0.085	824.9157089896964	20.51260096628154	0.1665909674815056	0.1665909674815056	0.1665909674815056	0.1665909674815056
0.0852	787.6063497590682	19.305087128058936	0.15390095439177506	0.15390095439177506	0.15390095439177506	0.15390095439177506
0.0854	751.9719903936846	18.168224425877504	0.14217289830378516	0.14217289830378516	0.14217289830378516	0.14217289830378516
0.0856	717.938054434494	17.09790657467146	0.13133425933900117	0.13133425933900117	0.13133425933900117	0.13133425933900117
0.0858	685.4332546534026	16.09026420155312	0.12131793701135837	0.12131793701135837	0.12131793701135837	0.12131793701135837
0.086	654.3894495391513	15.

0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	71135.87337728914	1049.1494429250065	4.343083380830239	4.343083380830239	4.343083380830239	4.343083380830239
0.0004	539530.8502579218	19387.80247162643	253.18850802589574	253.18850802589574	253.18850802589574	253.18850802589574
0.0006	1726345.3820654785	103689.28113323684	2626.983786013153	2626.983786013153	2626.983786013153	2626.983786013153
0.0008	3879551.530953204	333788.80678547476	13444.840681545975	13444.840681545975	13444.840681545975	13444.840681545975
0.001	7183716.609421044	813590.5129891956	46717.71432846784	46717.71432846784	46717.71432846784	46717.71432846784
0.0012	11768755.959500786	1663140.7202343794	127067.62461033904	127067.62461033904	127067.62461033904	127067.62461033904
0.0014	17717749.362329878	3011075.2623538887	291864.6500615481	291864.6500615481	291864.6500615481	291864.6500615481
0.0016	25073907.67272573	4987912.8074352145	592378.1453690883	592378.1453690883	592378.1453690883	592378.1453690883
0.0018	33846768.99477342	7720371.0

0.094	101.21143935119478	1.308410412308926	0.004569494865772889	0.004569494865772889	0.004569494865772889	0.004569494865772889
0.0942	96.56869806720537	1.2301959255124275	0.004215725620875911	0.004215725620875911	0.004215725620875911	0.004215725620875911
0.0944	92.13768123414077	1.1566344922401004	0.003889239918222342	0.003889239918222342	0.003889239918222342	0.003889239918222342
0.0946	87.90879608366802	1.087450738390665	0.003587942178675175	0.003587942178675175	0.003587942178675175	0.003587942178675175
0.0948	83.87288133419645	1.0223854946879252	0.003309897078364325	0.003309897078364325	0.003309897078364325	0.003309897078364325
0.095	80.02118793997796	0.9611948489737878	0.0030533173505244517	0.0030533173505244517	0.0030533173505244517	0.0030533173505244517
0.0952	76.3453606911831	0.9036492535499973	0.002816552511187477	0.002816552511187477	0.002816552511187477	0.002816552511187477
0.0954	72.83742062774289	0.8495326843952157	0.0025980784391348433	0.0025980784391348433	0.00259807843913